# Import packages

In [19]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir('/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/')

In [3]:
# Step 1: Install AIF360 and required dependencies
!pip install -q aif360
!pip install -q 'aif360[all]'
!pip install -q pandas numpy scikit-learn matplotlib seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.7/259.7 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from Baseline.Random import blind_random
from Baseline.xFAIR import *
from Baseline.FairSMOTE import Fair_Smote
from Baseline.FairC import FairC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
import pandas as pd
import numpy as np

/usr/local/lib/python3.11/dist-packages/inFairness/utils/ndcg.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  vect_normalized_discounted_cumulative_gain = vmap(
/usr/local/lib/python3.11/dist-packages/inFairness/utils/ndcg.py:48: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  monte_carlo_vect_ndcg = vmap(vect_normalized_discounted

In [5]:
import warnings
warnings.filterwarnings("ignore")

# Adult

In [ ]:
def load_and_process_adult_files(folder_path, output_folder=None):
    """
    Load all CSVs from a folder, drop non-numeric columns,
    and return processed DataFrames in a dictionary.

    Args:
        folder_path (str): Path to folder containing CSV files.
        output_folder (str, optional): Folder to save cleaned CSVs.

    Returns:
        dict: filename -> cleaned DataFrame
    """
    processed_dfs = {}
    if output_folder:
        os.makedirs(output_folder, exist_ok=True)

    for file in os.listdir(folder_path):
        if file.endswith(".csv"):
            file_path = os.path.join(folder_path, file)
            try:
                df = pd.read_csv(file_path)
                # Drop index column if it's present (commonly 'Unnamed: 0')
                index_cols = [col for col in df.columns if "unnamed" in col.lower()]
                df.drop(columns=index_cols, inplace=True)

                # Drop fnlwgt column if present
                df.drop(columns=['fnlwgt'], errors='ignore', inplace=True)

                # Rename "income" column "Probability"
                df.rename(columns={'income': 'Probability'}, inplace=True)

                # Keep only numeric columns
                numeric_df = df.select_dtypes(include=[np.number])
                processed_dfs[file] = numeric_df

                if output_folder:
                    output_path = os.path.join(output_folder, f"processed_{file}")
                    numeric_df.to_csv(output_path, index=False)

                print(f" Processed: {file} | Original shape: {df.shape} → Numeric shape: {numeric_df.shape}")
            except Exception as e:
                print(f" Failed to process {file}: {e}")

    return processed_dfs

In [ ]:
# Replace this with your actual folder containing CSVs
folder_path = "/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/datasets/Adult/Raw_Race"

processed = load_and_process_adult_files(folder_path, output_folder="/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/datasets/Adult/Processed_Race")

 Processed: Adult_Race_bias3.csv | Original shape: (45222, 14) → Numeric shape: (45222, 8)
 Processed: Adult_Race_bias2.csv | Original shape: (45222, 14) → Numeric shape: (45222, 8)
 Processed: Adult_Race_bias1.csv | Original shape: (45222, 14) → Numeric shape: (45222, 8)
 Processed: Adult_Race_bias4.csv | Original shape: (45222, 14) → Numeric shape: (45222, 8)
 Processed: Adult_Race_GenAlgo.csv | Original shape: (45222, 13) → Numeric shape: (45222, 13)


### Run the codes

In [ ]:
filenames = ['adult', 'GermanData']
keywords = {'adult': ['sex', 'race'],
            'GermanData': ['sex']
            }
base = RandomForestClassifier()
base2 = DecisionTreeRegressor()
for each in filenames:
    print("Filename: "+each)
    fname = each
    klist = keywords[fname]
    for keyword in klist:
        # Fair-SMOTE
        df1 = pd.read_csv("datasets/"+fname + "_processed.csv")
        result1 = Fair_Smote(df1, base, keyword=keyword, rep=1)
        print(result1)
        a, p, r, f, ao, eo, spd, di,fr, eopd = result1
        print("**"*50)
        print("Fair-SMOTE",fname, keyword)
        print("+Accuracy", np.mean(a))
        print("+Precision", np.mean(p))
        print("+Recall", np.mean(r))
        print("+F1", np.mean(f))
        print("-AOD", np.mean(ao))
        print("-EOD", np.mean(eo))
        print("-SPD", np.mean(spd))
        print("-DI", np.mean(di))
        print("-FR", np.mean(fr))
        print("-eopd", np.mean(eopd))

        # Random
        df1 = pd.read_csv("datasets/"+fname + "_processed.csv")
        result1 = blind_random(base, df1, keyword=keyword, rep=1)
        a, p, r, f, ao, eo, spd, di, fr, eopd = result1
        print("**" * 50)
        print("Random",fname, keyword)
        print("+Accuracy", np.mean(a))
        print("+Precision", np.mean(p))
        print("+Recall", np.mean(r))
        print("+F1", np.mean(f))
        print("-AOD", np.mean(ao))
        print("-EOD", np.mean(eo))
        print("-SPD", np.mean(spd))
        print("-DI", np.mean(di))
        print("-FR", np.mean(fr))
        print("-eopd", np.mean(eopd))

        # xFAIR
        df1 = pd.read_csv("datasets/"+fname + "_processed.csv")
        result1 = xFAIR(df1,base, base2, keyword=keyword, rep=1)
        a, p, r, f, ao, eo, spd, di, eopd = result1
        print("**" * 50)
        print("xFAIR",fname, keyword)
        print("+Accuracy", np.mean(a))
        print("+Precision", np.mean(p))
        print("+Recall", np.mean(r))
        print("+F1", np.mean(f))
        print("-AOD", np.mean(ao))
        print("-EOD", np.mean(eo))
        print("-SPD", np.mean(spd))
        print("-DI", np.mean(di))
        print("-eopd", np.mean(eopd))

        # FairC
        df1 = pd.read_csv("datasets/"+fname + "_processed.csv")
        result1 = FairC(df1, base, keyword=keyword, rep=1)
        a, p, r, f, ao, eo, spd, di, eopd = result1
        print("**"*50)
        print("FairC", fname, keyword)
        print("+Accuracy", np.mean(a))
        print("+Precision", np.mean(p))
        print("+Recall", np.mean(r))
        print("+F1", np.mean(f))
        print("-AOD", np.mean(ao))
        print("-EOD", np.mean(eo))
        print("-SPD", np.mean(spd))
        print("-DI", np.mean(di))
        print("-eopd", np.mean(eopd))


Filename: adult
Start generating samples...
Situational testing...
Round 1 finished.
Time 48.05037212371826
[[0.7894359709284471], [0.5495016611295681], [0.7023354564755838], [0.6165890027958993], [np.float64(0.07)], [np.float64(0.24)], [np.float64(0.21)], [np.float64(0.56)], [0.17872863138499334], [np.float64(0.24)]]
****************************************************************************************************
Fair-SMOTE adult sex
+Accuracy 0.7894359709284471
+Precision 0.5495016611295681
+Recall 0.7023354564755838
+F1 0.6165890027958993
-AOD 0.07
-EOD 0.24
-SPD 0.21
-DI 0.56
-FR 0.17872863138499334
-eopd 0.24
Round 1 finished. Time: 3.23 sec
****************************************************************************************************
Random adult sex
+Accuracy 0.8281297983416931
+Precision 0.6997635933806147
+Recall 0.50276008492569
+F1 0.5851247837904621
-AOD 0.0
-EOD 0.03
-SPD 0.11
-DI 0.52
-FR 0.07431671614290102
-eopd 0.03
Round 1 finished. Time: 4.79 sec
***********

In [ ]:
filenames = ['Adult_Census_orig',
             'Adult_Census_bias1', 'Adult_Census_bias2',  'Adult_Census_bias3',  'Adult_Census_bias4',
             'Adult_Census_GenAlgo',
             'Adult_Census_2000', 'Adult_Census_2005', 'Adult_Census_2010', 'Adult_Census_2015']
keywords = {
    'Adult_Census_orig': ['sex', 'race'],
    'Adult_Census_bias1': ['sex', 'race'],
    'Adult_Census_bias2': ['sex', 'race'],
    'Adult_Census_bias3': ['sex', 'race'],
    'Adult_Census_bias4': ['sex', 'race'],
    'Adult_Census_GenAlgo': ['sex', 'race'],
    'Adult_Census_2000': ['sex', 'race'],
    'Adult_Census_2005': ['sex', 'race'],
    'Adult_Census_2010': ['sex', 'race'],
    'Adult_Census_2015': ['sex', 'race']
}

base = RandomForestClassifier()
base2 = DecisionTreeRegressor()

results = []

for fname in filenames:
    for keyword in keywords[fname]:

        # --- Fair-SMOTE ---
        df = pd.read_csv(f"/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/datasets/Adult/Processed/processed_{fname}.csv")
        a, p, r, f, ao, eo, spd, di, fr, eopd = Fair_Smote(df, base, keyword=keyword, rep=1)
        results.append({
            'dataset': fname,
            'sensitive_feature': keyword,
            'technique': 'Fair-SMOTE',
            'acc': np.mean(a),
            'f1': np.mean(f),
            'prec': np.mean(p),
            'SPD': np.mean(spd),
            'EOPD': np.mean(eopd),
            'EOD': np.mean(eo),
            'AOD': np.mean(ao),
            'DI': np.mean(di),
            'FR': np.mean(fr)
        })

        # --- Random ---
        df = pd.read_csv(f"/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/datasets/Adult/Processed/processed_{fname}.csv")
        a, p, r, f, ao, eo, spd, di, fr, eopd = blind_random(base, df, keyword=keyword, rep=1)
        results.append({
            'dataset': fname,
            'sensitive_feature': keyword,
            'technique': 'Random',
            'acc': np.mean(a),
            'f1': np.mean(f),
            'prec': np.mean(p),
            'SPD': np.mean(spd),
            'EOPD': np.mean(eopd),
            'EOD': np.mean(eo),
            'AOD': np.mean(ao),
            'DI': np.mean(di),
            'FR': np.mean(fr)
        })

        # --- xFAIR ---
        df = pd.read_csv(f"/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/datasets/Adult/Processed/processed_{fname}.csv")
        a, p, r, f, ao, eo, spd, di, eopd = xFAIR(df, base, base2, keyword=keyword, rep=1)
        results.append({
            'dataset': fname,
            'sensitive_feature': keyword,
            'technique': 'xFAIR',
            'acc': np.mean(a),
            'f1': np.mean(f),
            'prec': np.mean(p),
            'SPD': np.mean(spd),
            'EOPD': np.mean(eopd),
            'EOD': np.mean(eo),
            'AOD': np.mean(ao),
            'DI': np.mean(di)
        })

        # --- FairC ---
        df = pd.read_csv(f"/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/datasets/Adult/Processed/processed_{fname}.csv")
        a, p, r, f, ao, eo, spd, di, eopd = FairC(df, base, keyword=keyword, rep=1)
        results.append({
            'dataset': fname,
            'sensitive_feature': keyword,
            'technique': 'FairC',
            'acc': np.mean(a),
            'f1': np.mean(f),
            'prec': np.mean(p),
            'SPD': np.mean(spd),
            'EOPD': np.mean(eopd),
            'EOD': np.mean(eo),
            'AOD': np.mean(ao),
            'DI': np.mean(di)
        })

# Convert to DataFrame
results_df = pd.DataFrame(results)

# Rearrange columns
cols = ['dataset', 'sensitive_feature', 'technique', 'acc', 'f1', 'prec', 'SPD', 'EOPD', 'EOD', 'AOD', 'DI', 'FR']
results_df = results_df[cols]

Start generating samples...
Situational testing...
Round 1 finished.
Time 48.50434732437134
Round 1 finished. Time: 3.24 sec
Round 1 finished. Time: 7.20 sec
Round 1 finished. Time: 6.64 sec
Start generating samples...
Situational testing...
Round 1 finished.
Time 77.18499851226807
Round 1 finished. Time: 3.66 sec
Round 1 finished. Time: 9.56 sec
Round 1 finished. Time: 6.72 sec
Start generating samples...
Situational testing...
Round 1 finished.
Time 58.923784255981445
Round 1 finished. Time: 3.24 sec
Round 1 finished. Time: 8.04 sec
Round 1 finished. Time: 6.62 sec
Start generating samples...
Situational testing...
Round 1 finished.
Time 87.89122152328491
Round 1 finished. Time: 3.16 sec
Round 1 finished. Time: 11.36 sec
Round 1 finished. Time: 6.29 sec
Start generating samples...
Situational testing...
Round 1 finished.
Time 65.81932473182678
Round 1 finished. Time: 3.11 sec
Round 1 finished. Time: 7.83 sec
Round 1 finished. Time: 6.38 sec
Start generating samples...
Situational tes

In [ ]:
filenames = ['Adult_Census_orig',
             'Adult_Race_bias1', 'Adult_Race_bias2',  'Adult_Race_bias3',  'Adult_Race_bias4',
             'Adult_Race_GenAlgo',
             'Adult_Census_2000', 'Adult_Census_2005', 'Adult_Census_2010', 'Adult_Census_2015']
keywords = {
    'Adult_Census_orig': ['sex', 'race'],
    'Adult_Race_bias1': ['sex', 'race'],
    'Adult_Race_bias2': ['sex', 'race'],
    'Adult_Race_bias3': ['sex', 'race'],
    'Adult_Race_bias4': ['sex', 'race'],
    'Adult_Race_GenAlgo': ['sex', 'race'],
    'Adult_Census_2000': ['sex', 'race'],
    'Adult_Census_2005': ['sex', 'race'],
    'Adult_Census_2010': ['sex', 'race'],
    'Adult_Census_2015': ['sex', 'race']
}

base = RandomForestClassifier()
base2 = DecisionTreeRegressor()

results = []

for fname in filenames:
    for keyword in keywords[fname]:

        # --- Fair-SMOTE ---
        df = pd.read_csv(f"/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/datasets/Adult/Processed_Race/processed_{fname}.csv")
        a, p, r, f, ao, eo, spd, di, fr, eopd = Fair_Smote(df, base, keyword=keyword, rep=1)
        results.append({
            'dataset': fname,
            'sensitive_feature': keyword,
            'technique': 'Fair-SMOTE',
            'acc': np.mean(a),
            'f1': np.mean(f),
            'prec': np.mean(p),
            'SPD': np.mean(spd),
            'EOPD': np.mean(eopd),
            'EOD': np.mean(eo),
            'AOD': np.mean(ao),
            'DI': np.mean(di),
            'FR': np.mean(fr)
        })

        # --- Random ---
        df = pd.read_csv(f"/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/datasets/Adult/Processed_Race/processed_{fname}.csv")
        a, p, r, f, ao, eo, spd, di, fr, eopd = blind_random(base, df, keyword=keyword, rep=1)
        results.append({
            'dataset': fname,
            'sensitive_feature': keyword,
            'technique': 'Random',
            'acc': np.mean(a),
            'f1': np.mean(f),
            'prec': np.mean(p),
            'SPD': np.mean(spd),
            'EOPD': np.mean(eopd),
            'EOD': np.mean(eo),
            'AOD': np.mean(ao),
            'DI': np.mean(di),
            'FR': np.mean(fr)
        })

        # --- xFAIR ---
        df = pd.read_csv(f"/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/datasets/Adult/Processed_Race/processed_{fname}.csv")
        a, p, r, f, ao, eo, spd, di, eopd = xFAIR(df, base, base2, keyword=keyword, rep=1)
        results.append({
            'dataset': fname,
            'sensitive_feature': keyword,
            'technique': 'xFAIR',
            'acc': np.mean(a),
            'f1': np.mean(f),
            'prec': np.mean(p),
            'SPD': np.mean(spd),
            'EOPD': np.mean(eopd),
            'EOD': np.mean(eo),
            'AOD': np.mean(ao),
            'DI': np.mean(di)
        })

        # --- FairC ---
        df = pd.read_csv(f"/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/datasets/Adult/Processed_Race/processed_{fname}.csv")
        a, p, r, f, ao, eo, spd, di, eopd = FairC(df, base, keyword=keyword, rep=1)
        results.append({
            'dataset': fname,
            'sensitive_feature': keyword,
            'technique': 'FairC',
            'acc': np.mean(a),
            'f1': np.mean(f),
            'prec': np.mean(p),
            'SPD': np.mean(spd),
            'EOPD': np.mean(eopd),
            'EOD': np.mean(eo),
            'AOD': np.mean(ao),
            'DI': np.mean(di)
        })

# Convert to DataFrame
results_df = pd.DataFrame(results)

# Rearrange columns
cols = ['dataset', 'sensitive_feature', 'technique', 'acc', 'f1', 'prec', 'SPD', 'EOPD', 'EOD', 'AOD', 'DI', 'FR']
results_df = results_df[cols]

Start generating samples...
Situational testing...
Round 1 finished.
Time 52.90917611122131
Round 1 finished. Time: 4.10 sec
Round 1 finished. Time: 7.06 sec
Round 1 finished. Time: 8.02 sec
Start generating samples...
Situational testing...
Round 1 finished.
Time 82.76175689697266
Round 1 finished. Time: 3.56 sec
Round 1 finished. Time: 10.76 sec
Round 1 finished. Time: 6.50 sec
Start generating samples...
Situational testing...
Round 1 finished.
Time 62.6642370223999
Round 1 finished. Time: 3.58 sec
Round 1 finished. Time: 8.98 sec
Round 1 finished. Time: 7.39 sec
Start generating samples...
Situational testing...
Round 1 finished.
Time 97.0471739768982
Round 1 finished. Time: 5.17 sec
Round 1 finished. Time: 9.84 sec
Round 1 finished. Time: 7.66 sec
Start generating samples...
Situational testing...
Round 1 finished.
Time 66.90365028381348
Round 1 finished. Time: 3.72 sec
Round 1 finished. Time: 6.97 sec
Round 1 finished. Time: 7.81 sec
Start generating samples...
Situational testin

In [ ]:
results_df

,dataset,sensitive_feature,technique,acc,f1,prec,SPD,EOPD,EOD,AOD,DI,FR
0,Adult_Census_orig,sex,Fair-SMOTE,0.785296,0.609413,0.566991,0.21,0.16,0.16,0.02,0.57,0.192814
1,Adult_Census_orig,sex,Random,0.813820,0.565979,0.694937,0.10,0.00,0.02,-0.01,0.47,0.111996
2,Adult_Census_orig,sex,xFAIR,0.813709,0.649470,0.599923,0.12,0.09,0.09,0.04,0.37,NaN
3,Adult_Census_orig,sex,FairC,0.826755,0.535979,0.772184,0.09,0.01,0.01,0.00,0.55,NaN
4,Adult_Census_orig,race,Fair-SMOTE,0.793035,0.625750,0.579201,0.16,0.12,0.12,0.01,0.48,0.161968
...,...,...,...,...,...,...,...,...,...,...,...,...
75,Adult_Census_2015,sex,FairC,0.830200,0.334639,0.565563,0.04,0.03,0.03,0.01,0.43,NaN
76,Adult_Census_2015,race,Fair-SMOTE,0.758200,0.455651,0.378884,0.11,0.16,0.16,0.04,0.39,0.191200
77,Adult_Census_2015,race,Random,0.817600,0.375770,0.476977,0.04,0.03,0.03,0.00,0.29,0.115800
78,Adult_Census_2015,race,xFAIR,0.800100,0.533053,0.459340,0.01,0.08,0.08,0.04,0.04,NaN


In [ ]:
# Save to CSV
results_df.to_csv("/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/Results/Adult_Race_FairCFairMask_results.csv", index=False)

# German

In [20]:
def load_and_process_german_files(folder_path, output_folder=None):
    """
    Load all CSVs from a folder, drop non-numeric columns,
    and return processed DataFrames in a dictionary.

    Args:
        folder_path (str): Path to folder containing CSV files.
        output_folder (str, optional): Folder to save cleaned CSVs.

    Returns:
        dict: filename -> cleaned DataFrame
    """
    processed_dfs = {}
    if output_folder:
        os.makedirs(output_folder, exist_ok=True)

    for file in os.listdir(folder_path):
        if file.endswith(".csv"):
            file_path = os.path.join(folder_path, file)
            try:
                df = pd.read_csv(file_path)
                # Create the new columns based on 'credit_history'
                df['credit_history=Delay'] = df['credit_history'].apply(lambda x: 1 if x == 'A33' else 0)
                df['credit_history=None/Paid'] = df['credit_history'].apply(lambda x: 1 if x == 'A30' or x == 'A31' or x == 'A32' else 0)
                df['credit_history=Other'] = df['credit_history'].apply(lambda x: 1 if x not in ['A30', 'A31', 'A32', 'A33'] else 0)

                # Create the new columns based on 'savings'
                df['savings=500+'] = df['savings'].apply(lambda x: 1 if x == 'A63' or x == 'A64' else 0)
                df['savings=<500'] = df['savings'].apply(lambda x: 1 if x == 'A61' or x == 'A62' else 0)
                df['savings=Unknown/None'] = df['savings'].apply(lambda x: 1 if x == 'A65' else 0)

                # Create the new columns based on 'employment'
                df['employment=1-4 years'] = df['employment'].apply(lambda x: 1 if x == 'A72' or x == 'A73' else 0)
                df['employment=4+ years'] = df['employment'].apply(lambda x: 1 if x == 'A74' or x == 'A75' else 0)
                df['employment=Unemployed'] = df['employment'].apply(lambda x: 1 if x == 'A71' else 0)

                df.rename(columns={'credit_risk': 'Probability'}, inplace=True)

                # Select and reorder the desired columns
                df_transformed = df[['sex', 'age', 'Probability', 'credit_history=Delay', 'credit_history=None/Paid',
                                    'credit_history=Other', 'savings=500+', 'savings=<500', 'savings=Unknown/None',
                                    'employment=1-4 years', 'employment=4+ years', 'employment=Unemployed']]

                # Keep only numeric columns
                numeric_df = df_transformed.select_dtypes(include=[np.number])
                processed_dfs[file] = numeric_df

                if output_folder:
                    output_path = os.path.join(output_folder, f"processed_{file}")
                    numeric_df.to_csv(output_path, index=False)

                print(f" Processed: {file} | Original shape: {df.shape} → Numeric shape: {numeric_df.shape}")
            except Exception as e:
                print(f" Failed to process {file}: {e}")

    return processed_dfs

In [21]:
# Replace this with your actual folder containing CSVs
folder_path = "/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/datasets/German/Raw"

processed = load_and_process_german_files(folder_path, output_folder="/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/datasets/German/Processed")

 Processed: German_orig.csv | Original shape: (1000, 32) → Numeric shape: (1000, 12)
 Processed: German_bias1.csv | Original shape: (1000, 32) → Numeric shape: (1000, 12)
 Processed: German_bias2.csv | Original shape: (1000, 32) → Numeric shape: (1000, 12)
 Processed: German_bias3.csv | Original shape: (1000, 32) → Numeric shape: (1000, 12)
 Processed: German_bias4.csv | Original shape: (1000, 32) → Numeric shape: (1000, 12)
 Processed: German_Age_bias1.csv | Original shape: (1000, 32) → Numeric shape: (1000, 12)
 Processed: German_Age_bias2.csv | Original shape: (1000, 32) → Numeric shape: (1000, 12)
 Processed: German_Age_bias3.csv | Original shape: (1000, 32) → Numeric shape: (1000, 12)
 Processed: German_Age_bias4.csv | Original shape: (1000, 32) → Numeric shape: (1000, 12)
 Failed to process German_GenAlgo_age.csv: 'savings'
 Failed to process German_GenAlgo_sex.csv: 'savings'


In [22]:
df = pd.read_csv("/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/datasets/German/Processed/processed_German_bias1.csv")

In [23]:
df['age'].value_counts()

,count
age,
1,800
0,200


In [24]:
filenames = ['German_orig',
             'German_bias1', 'German_bias2',  'German_bias3',  'German_bias4',
             'German_Age_bias1', 'German_Age_bias2',  'German_Age_bias3',  'German_Age_bias4',
             #'German_GenAlgo_sex', 'German_GenAlgo_age'
             ]
keywords = {
    'German_orig': ['sex', 'age'],
    'German_bias1': ['sex', 'age'],
    'German_bias2': ['sex', 'age'],
    'German_bias3': ['sex', 'age'],
    'German_bias4': ['sex', 'age'],
    'German_Age_bias1': ['sex', 'age'],
    'German_Age_bias2': ['sex', 'age'],
    'German_Age_bias3': ['sex', 'age'],
    'German_Age_bias4': ['sex', 'age'],
    #'German_GenAlgo_sex': ['sex', 'age'],
    #'German_GenAlgo_age': ['sex', 'age']
}

base = RandomForestClassifier()
base2 = DecisionTreeRegressor()

results = []

for fname in filenames:
    for keyword in keywords[fname]:
        print(f"Processing {fname} with sensitive feature: {keyword}")
        df = pd.read_csv(f"/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/datasets/German/Processed/processed_{fname}.csv")
        print(f"Shape of df before Fair-SMOTE: {df.shape}")
        # --- Fair-SMOTE ---
        try:
            a, p, r, f, ao, eo, spd, di, fr, eopd = Fair_Smote(df, base, keyword=keyword, rep=1)
            results.append({
                'dataset': fname,
                'sensitive_feature': keyword,
                'technique': 'Fair-SMOTE',
                'acc': np.mean(a),
                'f1': np.mean(f),
                'prec': np.mean(p),
                'SPD': np.mean(spd),
                'EOPD': np.mean(eopd),
                'EOD': np.mean(eo),
                'AOD': np.mean(ao),
                'DI': np.mean(di),
                'FR': np.mean(fr)
            })
        except ValueError as e:
            print(f"Error in Fair-SMOTE for {fname} with {keyword}: {e}")


        # --- Random ---
        df = pd.read_csv(f"/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/datasets/German/Processed/processed_{fname}.csv")
        print(f"Shape of df before Random: {df.shape}")
        a, p, r, f, ao, eo, spd, di, fr, eopd = blind_random(base, df, keyword=keyword, rep=1)
        results.append({
            'dataset': fname,
            'sensitive_feature': keyword,
            'technique': 'Random',
            'acc': np.mean(a),
            'f1': np.mean(f),
            'prec': np.mean(p),
            'SPD': np.mean(spd),
            'EOPD': np.mean(eopd),
            'EOD': np.mean(eo),
            'AOD': np.mean(ao),
            'DI': np.mean(di),
            'FR': np.mean(fr)
        })

        # --- xFAIR ---
        df = pd.read_csv(f"/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/datasets/German/Processed/processed_{fname}.csv")
        print(f"Shape of df before xFAIR: {df.shape}")
        if not (fname.startswith('German') and keyword == 'age' and df[keyword].nunique() <= 1):
            try:
                a, p, r, f, ao, eo, spd, di, eopd = xFAIR(df, base, base2, keyword=keyword, rep=1)
                results.append({
                    'dataset': fname,
                    'sensitive_feature': keyword,
                    'technique': 'xFAIR',
                    'acc': np.mean(a),
                    'f1': np.mean(f),
                    'prec': np.mean(p),
                    'SPD': np.mean(spd),
                    'EOPD': np.mean(eopd),
                    'EOD': np.mean(eo),
                    'AOD': np.mean(ao),
                    'DI': np.mean(di)
                })
            except ValueError as e:
                 print(f"Error in xFAIR for {fname} with {keyword}: {e}")
        else:
            print(f"Skipping xFAIR for {fname} with {keyword} due to single class in sensitive attribute.")


        # --- FairC ---
        df = pd.read_csv(f"/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/datasets/German/Processed/processed_{fname}.csv")
        print(f"Shape of df before FairC: {df.shape}")
        a, p, r, f, ao, eo, spd, di, eopd = FairC(df, base, keyword=keyword, rep=1)
        results.append({
            'dataset': fname,
            'sensitive_feature': keyword,
            'technique': 'FairC',
            'acc': np.mean(a),
            'f1': np.mean(f),
            'prec': np.mean(p),
            'SPD': np.mean(spd),
            'EOPD': np.mean(eopd),
            'EOD': np.mean(eo),
            'AOD': np.mean(ao),
            'DI': np.mean(di)
        })

# Convert to DataFrame
results_df = pd.DataFrame(results)

# Rearrange columns
cols = ['dataset', 'sensitive_feature', 'technique', 'acc', 'f1', 'prec', 'SPD', 'EOPD', 'EOD', 'AOD', 'DI', 'FR']
results_df = results_df[cols]

Processing German_orig with sensitive feature: sex
Shape of df before Fair-SMOTE: (1000, 12)
Start generating samples...
Situational testing...
Round 1 finished.
Time 1.370823860168457
Shape of df before Random: (1000, 12)
Round 1 finished. Time: 0.25 sec
Shape of df before xFAIR: (1000, 12)
Round 1 finished. Time: 0.27 sec
Shape of df before FairC: (1000, 12)
Round 1 finished. Time: 0.44 sec
Processing German_orig with sensitive feature: age
Shape of df before Fair-SMOTE: (1000, 12)
Start generating samples...
Situational testing...
Round 1 finished.
Time 1.602010726928711
Shape of df before Random: (1000, 12)
Round 1 finished. Time: 0.26 sec
Shape of df before xFAIR: (1000, 12)
Round 1 finished. Time: 0.27 sec
Shape of df before FairC: (1000, 12)
Round 1 finished. Time: 0.42 sec
Processing German_bias1 with sensitive feature: sex
Shape of df before Fair-SMOTE: (1000, 12)
Start generating samples...
Situational testing...
Round 1 finished.
Time 0.9085326194763184
Shape of df before Ra

In [25]:
results_df

,dataset,sensitive_feature,technique,acc,f1,prec,SPD,EOPD,EOD,AOD,DI,FR
0,German_orig,sex,Fair-SMOTE,0.615,0.675105,0.833333,0.00,-0.03,0.05,-0.04,0.01,0.050
1,German_orig,sex,Random,0.690,0.803797,0.725714,0.10,0.04,0.23,-0.10,0.11,0.045
2,German_orig,sex,xFAIR,0.565,0.641975,0.772277,0.16,0.09,0.28,-0.10,0.28,NaN
3,German_orig,sex,FairC,0.710,0.829412,0.712121,0.03,0.02,0.04,-0.01,0.03,NaN
4,German_orig,age,Fair-SMOTE,0.630,0.710938,0.791304,0.47,-0.35,0.76,0.20,0.99,0.615
...,...,...,...,...,...,...,...,...,...,...,...,...
67,German_Age_bias4,sex,FairC,0.755,0.860399,0.755000,0.00,0.00,0.00,0.00,0.00,NaN
68,German_Age_bias4,age,Fair-SMOTE,0.570,0.635593,0.852273,0.03,-0.01,0.09,-0.05,0.07,0.290
69,German_Age_bias4,age,Random,0.680,0.785235,0.780000,0.31,-0.27,0.41,0.07,0.46,0.320
70,German_Age_bias4,age,xFAIR,0.530,0.620968,0.793814,0.11,0.16,0.16,0.10,0.22,NaN


In [26]:
# Save to CSV
results_df.to_csv("/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/Results/German_FairCFairMask_results.csv", index=False)

# Student

In [ ]:
def load_and_process_student_files(folder_path, output_folder=None):
    """
    Load all CSVs from a folder, drop non-numeric columns,
    and return processed DataFrames in a dictionary.

    Args:
        folder_path (str): Path to folder containing CSV files.
        output_folder (str, optional): Folder to save cleaned CSVs.

    Returns:
        dict: filename -> cleaned DataFrame
    """
    processed_dfs = {}
    if output_folder:
        os.makedirs(output_folder, exist_ok=True)

    for file in os.listdir(folder_path):
        if file.endswith(".csv"):
            file_path = os.path.join(folder_path, file)
            try:
                df = pd.read_csv(file_path)
                # Drop index column if it's present (commonly 'Unnamed: 0')
                index_cols = [col for col in df.columns if "unnamed" in col.lower()]
                df.drop(columns=index_cols, inplace=True)

                # Rename "income" column "Probability"
                df.rename(columns={'dropout': 'Probability'}, inplace=True)

                # Keep only numeric columns
                numeric_df = df.select_dtypes(include=[np.number])
                processed_dfs[file] = numeric_df

                if output_folder:
                    output_path = os.path.join(output_folder, f"processed_{file}")
                    numeric_df.to_csv(output_path, index=False)

                print(f" Processed: {file} | Original shape: {df.shape} → Numeric shape: {numeric_df.shape}")
            except Exception as e:
                print(f" Failed to process {file}: {e}")

    return processed_dfs

In [ ]:
# Replace this with your actual folder containing CSVs
folder_path = "/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/datasets/Student/Raw"

processed = load_and_process_student_files(folder_path, output_folder="/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/datasets/Student/Processed")

 Processed: Student_GenAlgo_Sex.csv | Original shape: (4424, 37) → Numeric shape: (4424, 37)
 Processed: Student_orig.csv | Original shape: (4424, 37) → Numeric shape: (4424, 37)
 Processed: Student_bias2.csv | Original shape: (4424, 37) → Numeric shape: (4424, 37)
 Processed: Student_bias3.csv | Original shape: (4424, 37) → Numeric shape: (4424, 37)
 Processed: Student_bias4.csv | Original shape: (4424, 37) → Numeric shape: (4424, 37)
 Processed: Student_Age_bias1.csv | Original shape: (4424, 37) → Numeric shape: (4424, 37)
 Processed: Student_Age_bias2.csv | Original shape: (4424, 37) → Numeric shape: (4424, 37)
 Processed: Student_Age_bias3.csv | Original shape: (4424, 37) → Numeric shape: (4424, 37)
 Processed: Student_Age_bias4.csv | Original shape: (4424, 37) → Numeric shape: (4424, 37)
 Processed: Student_GenAlgo_Age.csv | Original shape: (4424, 37) → Numeric shape: (4424, 37)
 Processed: Student_bias1.csv | Original shape: (4424, 37) → Numeric shape: (4424, 37)


In [ ]:
filenames = ['Student_orig',
             'Student_bias1', 'Student_bias2',  'Student_bias3',  'Student_bias4',
             'Student_Age_bias1', 'Student_Age_bias2',  'Student_Age_bias3',  'Student_Age_bias4',
             'Student_GenAlgo_Sex', 'Student_GenAlgo_Age']
keywords = {
    'Student_orig': ['Gender', 'age'],
    'Student_bias1': ['Gender', 'age'],
    'Student_bias2': ['Gender', 'age'],
    'Student_bias3': ['Gender', 'age'],
    'Student_bias4': ['Gender', 'age'],
    'Student_Age_bias1': ['Gender', 'age'],
    'Student_Age_bias2': ['Gender', 'age'],
    'Student_Age_bias3': ['Gender', 'age'],
    'Student_Age_bias4': ['Gender', 'age'],
    'Student_GenAlgo_Sex': ['Gender', 'age'],
    'Student_GenAlgo_Age': ['Gender', 'age'],
}

base = RandomForestClassifier()
base2 = DecisionTreeRegressor()

results = []

for fname in filenames:
    for keyword in keywords[fname]:

        # --- Fair-SMOTE ---
        df = pd.read_csv(f"/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/datasets/Student/Processed/processed_{fname}.csv")
        a, p, r, f, ao, eo, spd, di, fr, eopd = Fair_Smote(df, base, keyword=keyword, rep=1)
        results.append({
            'dataset': fname,
            'sensitive_feature': keyword,
            'technique': 'Fair-SMOTE',
            'acc': np.mean(a),
            'f1': np.mean(f),
            'prec': np.mean(p),
            'SPD': np.mean(spd),
            'EOPD': np.mean(eopd),
            'EOD': np.mean(eo),
            'AOD': np.mean(ao),
            'DI': np.mean(di),
            'FR': np.mean(fr)
        })

        # --- Random ---
        df = pd.read_csv(f"/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/datasets/Student/Processed/processed_{fname}.csv")
        a, p, r, f, ao, eo, spd, di, fr, eopd = blind_random(base, df, keyword=keyword, rep=1)
        results.append({
            'dataset': fname,
            'sensitive_feature': keyword,
            'technique': 'Random',
            'acc': np.mean(a),
            'f1': np.mean(f),
            'prec': np.mean(p),
            'SPD': np.mean(spd),
            'EOPD': np.mean(eopd),
            'EOD': np.mean(eo),
            'AOD': np.mean(ao),
            'DI': np.mean(di),
            'FR': np.mean(fr)
        })

        # --- xFAIR ---
        df = pd.read_csv(f"/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/datasets/Student/Processed/processed_{fname}.csv")
        a, p, r, f, ao, eo, spd, di, eopd = xFAIR(df, base, base2, keyword=keyword, rep=1)
        results.append({
            'dataset': fname,
            'sensitive_feature': keyword,
            'technique': 'xFAIR',
            'acc': np.mean(a),
            'f1': np.mean(f),
            'prec': np.mean(p),
            'SPD': np.mean(spd),
            'EOPD': np.mean(eopd),
            'EOD': np.mean(eo),
            'AOD': np.mean(ao),
            'DI': np.mean(di)
        })

        # --- FairC ---
        df = pd.read_csv(f"/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/datasets/Student/Processed/processed_{fname}.csv")
        a, p, r, f, ao, eo, spd, di, eopd = FairC(df, base, keyword=keyword, rep=1)
        results.append({
            'dataset': fname,
            'sensitive_feature': keyword,
            'technique': 'FairC',
            'acc': np.mean(a),
            'f1': np.mean(f),
            'prec': np.mean(p),
            'SPD': np.mean(spd),
            'EOPD': np.mean(eopd),
            'EOD': np.mean(eo),
            'AOD': np.mean(ao),
            'DI': np.mean(di)
        })

# Convert to DataFrame
results_df = pd.DataFrame(results)

# Rearrange columns
cols = ['dataset', 'sensitive_feature', 'technique', 'acc', 'f1', 'prec', 'SPD', 'EOPD', 'EOD', 'AOD', 'DI', 'FR']
results_df = results_df[cols]

Start generating samples...
Situational testing...
Round 1 finished.
Time 7.207418441772461
Round 1 finished. Time: 0.99 sec
Round 1 finished. Time: 1.86 sec
Round 1 finished. Time: 2.29 sec
Start generating samples...
Situational testing...
Round 1 finished.
Time 9.629606246948242
Round 1 finished. Time: 1.18 sec
Round 1 finished. Time: 1.90 sec
Round 1 finished. Time: 1.61 sec
Start generating samples...
Situational testing...
Round 1 finished.
Time 3.8742525577545166
Round 1 finished. Time: 0.91 sec
Round 1 finished. Time: 1.35 sec
Round 1 finished. Time: 1.70 sec
Start generating samples...
Situational testing...
Round 1 finished.
Time 8.05123496055603
Round 1 finished. Time: 0.89 sec
Round 1 finished. Time: 1.36 sec
Round 1 finished. Time: 1.72 sec
Start generating samples...
Situational testing...
Round 1 finished.
Time 6.698770761489868
Round 1 finished. Time: 0.90 sec
Round 1 finished. Time: 1.32 sec
Round 1 finished. Time: 1.55 sec
Start generating samples...
Situational testi

In [ ]:
results_df

,dataset,sensitive_feature,technique,acc,f1,prec,SPD,EOPD,EOD,AOD,DI,FR
0,Student_orig,Gender,Fair-SMOTE,0.856497,0.892646,0.859935,0.26,-0.08,0.08,0.00,0.50,0.016949
1,Student_orig,Gender,Random,0.859887,0.896667,0.852615,0.26,-0.10,0.10,-0.02,0.49,0.009040
2,Student_orig,Gender,xFAIR,0.970621,0.978405,0.976783,0.18,-0.02,0.02,-0.01,0.32,NaN
3,Student_orig,Gender,FairC,0.877966,0.913323,0.882171,0.17,-0.06,0.06,-0.01,0.28,NaN
4,Student_orig,age,Fair-SMOTE,0.859887,0.894558,0.866557,0.27,0.02,0.11,-0.04,0.36,0.012429
5,Student_orig,age,Random,0.854237,0.892410,0.849206,0.28,0.05,0.10,-0.02,0.35,0.002260
6,Student_orig,age,xFAIR,0.963842,0.973510,0.968699,0.31,0.02,0.02,0.02,0.41,NaN
7,Student_orig,age,FairC,0.868927,0.906300,0.880691,0.29,0.01,0.15,-0.07,0.37,NaN
8,Student_bias1,Gender,Fair-SMOTE,0.849718,0.858058,0.827160,0.09,-0.09,0.09,-0.06,0.19,0.020339
9,Student_bias1,Gender,Random,0.846328,0.854390,0.826087,0.10,-0.09,0.09,-0.04,0.22,0.009040


In [ ]:
# Save to CSV
results_df.to_csv("/content/drive/MyDrive/UCL_MSc_AI/UCLFinalProj/FinalProj_Main/Results/Student_FairCFairMask_results.csv", index=False)

In [ ]:
display(results_df)